## MIDS UC Berkeley, Machine Learning at Scale
## DATSCIW261 ASSIGNMENT #1  

James Gray | jamesgray@ischool.berkeley.edu

Date Submitted: May x, 2016



## HW1.0.0. 

Define big data. Provide an example of a big data problem in your domain of expertise. 


Conceptually, Big Data is defined as data that are so large, complex and speed that traditional applications are not adequate to deliver solutions.  Big Data is often described by three or four V's.  Volume often distinquishes Big Data as datasets that reach the petabyte (10^15) or zettbyte (10^21) scale when compared to traditional data processing applications in the terabytes (10^12). Variety includes both structured and unstructed data such as video, text, JSON, images unlike traditional systems that handled well formed data in rows and columns. Velocity includes the ability of Big Data solutions to handle very fast streaming data from sources such as the social web or machine-to-machine scenarios in the Internet of Things realm.  Veracity is the uncertainty of data and ability to manage and enforce data quality.  All of these attributes make traditional data processing systems such as databases completely inadequate.

Example here

## HW1.0.1.

In 500 words (English or pseudo code or a combination) describe how to estimate the bias, the variance, the irreduciable error for a test dataset T when using polynomial regression models of degree 1,2,3,4,5 are considered. How would you select a model?


## Instructions for Spam Filter using Naive Bayes Classifier

In the remainder of this assignment you will produce a spam filter that is backed by a multinomial naive Bayes classifier b (see http://nlp.stanford.edu/IR-book/html/htmledition/properties-of-naive-bayes-1.html),
which counts words in parallel via a unix, poor-man's map-reduce framework.  

The data you will use is a curated subset of the Enron email corpus(whose details you may find in the file enronemail_README.txt in the directory surrounding these instructions).

In this directory you will also find starter code (pNaiveBayes.sh),(similar to the pGrepCount.sh code that was presented in this weeks lectures), which will be used as control script to a python mapper and reducer that you will supply at several stages. Doing some exploratory data analysis you will see (with this very small dataset) the following\:





In [ ]:
! wc -l enronemail_1h.txt  #100 email records

! cut -f2 -d$'\t' enronemail_1h.txt|wc  #extract second field which is SPAM flag

! cut -f2 -d$'\t' enronemail_1h.txt|head

! head -n 100 enronemail_1h.txt|tail -1|less #an example SPAM email record


      99 enronemail_1h.txt
     100     100     200
0
0
0
0
0
0
0
0
1
1
0018.2003-12-18.GP      1        await your response    " dear partner,  we are  a team of government officials that belong to an eight-man committee in the pres idential cabinet as well as the senate.  at the moment, we will be requiring you r assistance in a matter that involves investment of monies, which we intend to  transfer to your account, upon clarification and a workable agreement reached in  consummating the project with you. based on a recommendation from an associate  concerning your integrity, loyalty and understanding, we deemed it necessary to  contact you accordingly. all arrangements in relation to this investment initiat ive, as well as the initial capital for its take off has been tactically set asi de to commence whatever business you deemed fit, that will turn around profit fa vourably. we request you immediately contact us if you will be favorably dispose d to act as a partner in t

## HW1.1.

Read through the provided control script (pNaiveBayes.sh) and all of its comments. When you are comfortable with their purpose and function, respond to the remaining homework questions below. A simple cell in the notebook with a print statement with  a "done" string will suffice here.

In [ ]:
## pNaiveBayes.sh
## Author: Jake Ryland Williams
## Usage: pNaiveBayes.sh m wordlist
## Input:
##       m = number of processes (maps), e.g., 4
##       wordlist = a space-separated list of words in quotes, e.g., "the and of"
##
## Instructions: Read this script and its comments closely.
##               Do your best to understand the purpose of each command,
##               and focus on how arguments are supplied to mapper.py/reducer.py,
##               as this will determine how the python scripts take input.
##               When you are comfortable with the unix code below,
##               answer the questions on the LMS for HW1 about the starter code.

## collect user input
m=$1 ## the number of parallel processes (maps) to run
wordlist=$2 ## if set to "*", then all words are used

## a test set data of 100 messages
data="enronemail_1h.txt" 

## the full set of data (33746 messages)
# data="enronemail.txt" 

## 'wc' determines the number of lines in the data
## 'perl -pe' regex strips the piped wc output to a number
linesindata=`wc -l $data | perl -pe 's/^.*?(\d+).*?$/$1/'`

## determine the lines per chunk for the desired number of processes
linesinchunk=`echo "$linesindata/$m+1" | bc` 

## split the original file into chunks by line
split -l $linesinchunk $data $data.chunk.

## assign python mappers (mapper.py) to the chunks of data
## and emit their output to temporary files
for datachunk in $data.chunk.*; do
    ## feed word list to the python mapper here and redirect STDOUT to a temporary file on disk
    ####
    ####
    ./mapper.py $datachunk "$wordlist" > $datachunk.counts &
    ####
    ####
done
## wait for the mappers to finish their work
wait

## 'ls' makes a list of the temporary count files
## 'perl -pe' regex replaces line breaks with spaces
countfiles=`\ls $data.chunk.*.counts | perl -pe 's/\n/ /'`

## feed the list of countfiles to the python reducer and redirect STDOUT to disk
####
####
./reducer.py $countfiles > $data.output
####
####

## clean up the data chunks and temporary count files
\rm $data.chunk.*


## HW1.2. 

Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will determine the number of occurrences of a single, user-specified word. Examine the word “assistance” and report your results.  To do so, make sure that:
   
* mapper.py counts all occurrences of a single word, and
* reducer.py collates the counts of the single word.


## HW1.3. 

Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will classify the email messages by a single, user-specified word using the multinomial Naive Bayes Formulation. Examine the word “assistance” and report your results.
   
To do so, make sure that:
   
   - mapper.py and
   - reducer.py 

performs a single word Naive Bayes classification. For multinomial Naive Bayes, the Pr(X=“assistance”|Y=SPAM) is calculated as follows:

the number of times “assistance” occurs in SPAM labeled documents / the number of words in documents labeled SPAM 

NOTE if  “assistance” occurs 5 times in all of the documents Labeled SPAM, and the length in terms of the number of words in all documents labeld as SPAM (when concatenated) is 1,000. Then Pr(X=“assistance”|Y=SPAM) = 5/1000. Note this is a multinomial estimated of the class conditional for a Naive Bayes Classifier. No smoothing is needed in this HW.


## HW1.4. 

Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will classify the email messages by a list of one or more user-specified words.

Examine the words “assistance”, “valium”, and “enlargementWithATypo” and report your results (accuracy). To do so, make sure that:

* mapper.py counts all occurrences of a list of words, and
* reducer.py 

performs the multiple-word multinomial Naive Bayes classification via the chosen list. No smoothing is needed in this HW.


## HW1.5. 

Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will classify the email messages by all words present.

To do so, make sure that:

* mapper.py counts all occurrences of all words, and
* reducer.py performs a word-distribution-wide Naive Bayes classification.

## HW1.6 

Benchmark your code with the Python SciKit-Learn implementation of multinomial Naive Bayes.

It always a good idea to test your solutions against publicly available libraries such as SciKit-Learn, The Machine Learning toolkit available in Python. In this exercise, we benchmark ourselves against the SciKit-Learn implementation of multinomial Naive Bayes.  For more information on this implementation see: http://scikit-learn.org/stable/modules/naive_bayes.html more  

Training error = misclassification rate with respect to a training set. It is more formally defined here:

Let DF represent the training set in the following:
Err(Model, DF) = |{(X, c(X)) ∈ DF : c(X) != Model(x)}|   / |DF|

Where || denotes set cardinality; c(X) denotes the class of the tuple X in DF; and Model(X) denotes the class inferred by the Model “Model”

In this exercise, please complete the following:

* Run the Multinomial Naive Bayes algorithm (using default settings) from SciKit-Learn over the same training data used in HW1.5 and report the Training error (please note some data preparation might be needed to get the Multinomial Naive Bayes algorithm from SkiKit-Learn to run over this dataset)
* Please prepare a table to present your results
* Explain/justify any differences in terms of training error rates over the dataset in HW1.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus the Multinomial Naive Bayes implementation in SciKit-Learn (Hint: smoothing, which we will discuss in next lecture)
* Discuss the performance differences in terms of training error rates over the dataset in HW1.5 between the  Multinomial Naive Bayes implementation in SciKit-Learn with the  Bernoulli Naive Bayes implementation in SciKit-Learn

